In [1]:
#default_exp pleiades

# NovaSAR

> Working through stac item metadata parsing, etc. for NovaSAR datasets (data already prepped to COG).

In [2]:
#hide
# %load_ext autoreload
# %autoreload 2

In [3]:
# from nbdev.showdoc import *

In [3]:
import os
import boto3 

import numpy as np
from datetime import datetime
import json

import xmltodict
import pystac
from pystac import StacIO
from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.sar import FrequencyBand, Polarization, ObservationDirection
from pystac.extensions.sar import SarExtension
import rasterio
from shapely.geometry import Polygon

import ipynb

from ipynb.fs.defs.utils import *

## **STAC metadata**: ***core & extensions***

With the cogs above hosted object storage we can go create some tools for building a STAC *Collection* from them, to be used within in **insert nb** to build a *Catalog* of different Catapult *Collections*. As per the nb these initially sit within a *static* STAC alongside the actual datasets on the object storage. However we plan on hosting via a STAC compliant API - probs [pygeoapi](https://pygeoapi.io/). First define bucket and imagery directory

In [5]:
s3_bucket = 'public-eo-data'
img_dir = 'novasar_uk_test/'

obj_paths_list = s3_list_objects_paths(s3_bucket, img_dir)

# obj_paths_list

In [6]:
# get unique Item / scene names (third dir from path)

scene_names = list(np.unique([i.split('/')[2] for i in obj_paths_list if 'uk_novasar_scansar_20m' in i or 'uk_novasar_stripmap_6m' in i]))[1:]

scene_names[:5]

['NovaSAR_01_10135_slc_11_200215_094557_HH_1_ML_TC_TF_cog',
 'NovaSAR_01_10135_slc_11_200215_094601_HH_2_ML_TC_TF_cog',
 'NovaSAR_01_10387_slc_11_200307_101114_HH_1_ML_TC_TF_cog',
 'NovaSAR_01_10387_slc_11_200307_101118_HH_2_ML_TC_TF_cog',
 'NovaSAR_01_10387_slc_11_200307_101122_HH_3_ML_TC_TF_cog']

### **Individual example for iteration**

Can just work with one

In [7]:
scene_name = scene_names[0]
scene_name

'NovaSAR_01_10135_slc_11_200215_094557_HH_1_ML_TC_TF_cog'

and objects associated with that scene

In [8]:
scene_obj_paths = [i for i in obj_paths_list if scene_name in i]
list(scene_obj_paths)

['novasar_uk_test/uk_novasar_stripmap_6m/NovaSAR_01_10135_slc_11_200215_094557_HH_1_ML_TC_TF_cog/NovaSAR_01_10135_slc_11_200215_094557_HH_1_Gamma0_Intensity_HH_db.tif',
 'novasar_uk_test/uk_novasar_stripmap_6m/NovaSAR_01_10135_slc_11_200215_094557_HH_1_ML_TC_TF_cog/original_metadata.xml',
 'novasar_uk_test/uk_novasar_stripmap_6m/NovaSAR_01_10135_slc_11_200215_094557_HH_1_ML_TC_TF_cog/process_metadata.dim',
 'novasar_uk_test/uk_novasar_stripmap_6m/NovaSAR_01_10135_slc_11_200215_094557_HH_1_ML_TC_TF_cog/thumbnail.jpg']

set our own I/O for pystac

In [9]:
#export
pystac_setIO()

### **Functions for** ***Item*** **metadata**

In [10]:
#export
def novasar_get_dt(scene_name):
    return datetime.strptime('_'.join(scene_name.split('_')[5:7]), '%y%m%d_%H%M%S')

In [11]:
novasar_get_dt(scene_name)

datetime.datetime(2020, 2, 15, 9, 45, 57)

In [12]:
#export
def novasar_parsemeta(scene_name, scene_obj_paths):
    meta_path = [i for i in scene_obj_paths if (i.endswith('.dim'))][0]
    return xmltodict.parse(pystac.STAC_IO.read_text(create_uri(meta_path)))

# TODO: link to SeDAS API

In [13]:
# meta = novasar_parsemeta(scene_name, scene_obj_paths)

# orbit_data = meta['Dimap_Document']['Dataset_Sources']['MDElem']['MDElem'][1]['MDElem']['MDElem'][2]['MDATTR'][0]['#text'] #this seems daft

In [14]:
def novasar_get_crs_and_bbox(raster_uri):
    """
    BBOX list, geometry shapely and rasterio crs from
    URI of COG.
    nb: footprint currently same as bbo.
    """
    with rasterio.open(raster_uri) as ds:
        bounds = ds.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = [[
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.top],
            [bounds.right, bounds.bottom],
            [bounds.left, bounds.bottom]
        ]] 
        return bbox, {'type': 'Polygon', 'coordinates': footprint}, ds.crs #changed format of geometry

In [15]:
def novasar_get_instrument_mode(scene_obj_paths): 
    return '_'.join(scene_obj_paths[0].split('/')[1].split('_')[2:]) # any object path

In [16]:
novasar_get_instrument_mode(scene_obj_paths)

'stripmap_6m'

In [17]:
bbox, g, epsg = novasar_get_crs_and_bbox(create_uri([i for i in scene_obj_paths if i.endswith('.tif')][0]))
bbox, g, epsg.to_dict()['init'][5:]

([-1.0478259576205007,
  50.397680132459946,
  -0.6671061061552759,
  50.6936401639443],
 {'type': 'Polygon',
  'coordinates': [[[-1.0478259576205007, 50.397680132459946],
    [-1.0478259576205007, 50.6936401639443],
    [-0.6671061061552759, 50.6936401639443],
    [-0.6671061061552759, 50.397680132459946],
    [-1.0478259576205007, 50.397680132459946]]]},
 '4326')

In [18]:
#export
def novasar_create_item(scene_name, scene_obj_paths):

#     meta = pleiades_parsemeta(scene_name, scene_obj_paths)

#     crs = pleiades_get_crs(meta)

    bbox, g, epsg = novasar_get_crs_and_bbox(create_uri([i for i in scene_obj_paths if i.endswith('.tif')][0]))

    item = pystac.Item(id=scene_name,
                      datetime=novasar_get_dt(scene_name),
                      geometry=g,
                      bbox=bbox,
                      properties={})

    # need to add func for res of novasar
#     item.common_metadata.gsd = pleiades_get_gsd(meta)

#    item.ext.enable('projection')
#    item.ext.projection.epsg = int(epsg.to_dict()['init'][5:])
    
#    item.ext.enable('sar')
#    item.ext.sar.instrument_mode = novasar_get_instrument_mode(scene_obj_paths)
#    item.ext.sar.frequency_band = FrequencyBand('S')

    SarExtension.add_to(item)

    sar_ext = SarExtension.ext(item)
    sar_ext.frequency_band = FrequencyBand('S')
    sar_ext.instrument_mode = novasar_get_instrument_mode(scene_obj_paths)

    ProjectionExtension.add_to(item)

    proj_ext = ProjectionExtension.ext(item)#
    proj_ext.epsg = int(epsg.to_dict()['init'][5:])
    
    # item.ext.sar.orbit = orbit_data (no such extension)

    return item

In [19]:
item = novasar_create_item(scene_name, scene_obj_paths)

In [139]:
type(SarExtension.ext)

method

In [20]:
item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'NovaSAR_01_10135_slc_11_200215_094557_HH_1_ML_TC_TF_cog',
 'properties': {'sar:frequency_band': 'S',
  'sar:instrument_mode': 'stripmap_6m',
  'proj:epsg': 4326,
  'datetime': '2020-02-15T09:45:57Z'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-1.0478259576205007, 50.397680132459946],
    [-1.0478259576205007, 50.6936401639443],
    [-0.6671061061552759, 50.6936401639443],
    [-0.6671061061552759, 50.397680132459946],
    [-1.0478259576205007, 50.397680132459946]]]},
 'links': [],
 'assets': {},
 'bbox': [-1.0478259576205007,
  50.397680132459946,
  -0.6671061061552759,
  50.6936401639443],
 'stac_extensions': ['https://stac-extensions.github.io/sar/v1.0.0/schema.json',
  'https://stac-extensions.github.io/projection/v1.0.0/schema.json']}

In [134]:
# item.validate() # won't work as need polarisation (added below)

### **Functions for** ***Asset*** **metadata**

Once we have *Item* level metadata we can add the actual *Assets*. There are a few constants used within these functions at the *Item* level (i.e. band info) and criteria for finding within the object paths.

In [21]:
# first, get href(s) for asset(s) in scene

# cog_hrefs = list(map(create_uri, [i for i in obj_paths_list if scene_name in i if i.endswith('.tif')])) # this would create href for all tifs in folder

cog_href = create_uri([i for i in scene_obj_paths if i.endswith('.tif')][0])

if len([i for i in scene_obj_paths if i.endswith('.jpg')])==0: # messy workaround to account for some scenes not having thumbnails
    thumb_href = ''
    
else:
    thumb_href = create_uri([i for i in scene_obj_paths if i.endswith('.jpg')][0])

asset_hrefs = [cog_href, thumb_href]

In [22]:
def novasar_get_pol(asset_path):
    asset_name = os.path.basename(asset_path)
    if '_VV_' in asset_name:
        pol = Polarization('VV')
    elif '_HH_' in asset_name:
        pol = Polarization('HH')
    return pol

In [23]:
novasar_get_pol(cog_href)

<Polarization.HH: 'HH'>

In [24]:
def novasar_get_prod_type(asset_path): #TODO: generalise for multiple assets ??
    asset_name = os.path.basename(asset_path)
    if 'Gamma0' in asset_path:
        prod = 'gamma0_db'
    return prod

In [25]:
novasar_get_prod_type(cog_href)

'gamma0_db'

In [26]:
[novasar_get_pol(cog_href)]

[<Polarization.HH: 'HH'>]

In [85]:
def add_assets_novasar(item, asset_paths):
    
    for asset_path in asset_paths:
        
        if asset_path.endswith('.tif'):
            # print(asset_path)
            item.add_asset('cog', pystac.Asset(href = asset_path, media_type = pystac.MediaType.COG))
            
            # item.ext.sar.polarizations = [Polarization(novasar_get_pol(asset_path))]
            # item.ext.sar.product_type = novasar_get_prod_type(os.path.basename(asset_path))
            
            sar_ext = SarExtension.ext(item)
            
            sar_ext.polarizations = [novasar_get_pol(asset_path)]
            sar_ext.product_type = novasar_get_prod_type(os.path.basename(asset_path))
     
        elif asset_path.endswith('.jpg'):
            # print(asset_path)
            item.add_asset('thumbnail', pystac.Asset(href = asset_path, media_type= pystac.MediaType.JPEG))       

            
    return item

In [86]:
add_assets_novasar(item, asset_hrefs)

<Item id=NovaSAR_01_10135_slc_11_200215_094557_HH_1_ML_TC_TF_cog>

In [89]:
item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'NovaSAR_01_10135_slc_11_200215_094557_HH_1_ML_TC_TF_cog',
 'properties': {'sar:frequency_band': 'S',
  'sar:instrument_mode': 'stripmap_6m',
  'proj:epsg': 4326,
  'datetime': '2020-02-15T09:45:57Z',
  'sar:polarizations': ['HH'],
  'sar:product_type': 'gamma0_db'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-1.0478259576205007, 50.397680132459946],
    [-1.0478259576205007, 50.6936401639443],
    [-0.6671061061552759, 50.6936401639443],
    [-0.6671061061552759, 50.397680132459946],
    [-1.0478259576205007, 50.397680132459946]]]},
 'links': [],
 'assets': {'cog': {'href': 'http://s3-uk-1.sa-catapult.co.uk/public-eo-data/novasar_uk_test/uk_novasar_stripmap_6m/NovaSAR_01_10135_slc_11_200215_094557_HH_1_ML_TC_TF_cog/NovaSAR_01_10135_slc_11_200215_094557_HH_1_Gamma0_Intensity_HH_db.tif',
   'type': <MediaType.COG: 'image/tiff; application=geotiff; profile=cloud-optimized'>},
  'thumbnail': {'href': 'http://s3-uk-1.sa-catapul

In [90]:
item.validate()

['https://schemas.stacspec.org/v1.0.0/item-spec/json-schema/item.json',
 'https://stac-extensions.github.io/sar/v1.0.0/schema.json',
 'https://stac-extensions.github.io/projection/v1.0.0/schema.json']

In [158]:
SarExtension.get_schema_uri()

'https://stac-extensions.github.io/sar/v1.0.0/schema.json'

## Make Collection

Now gather 6m and 20m resolution assets together into collections

In [106]:
def create_novasar_collection(novasar_dir, bucket=s3_bucket): #, n=None):
    
    collection_id = f'novasar_{novasar_dir[27:]}'
    collection_title = 'NovaSAR test'
    collection_description = '''### NovaSAR test STACs

    A collection of NovaSAR COGs stored in Catapult object storage
    '''
    
    # initially arbitrary as updated later
    spatial_extent = pystac.SpatialExtent([[-180, 90, 180, 90]])
    temporal_extent = pystac.TemporalExtent([[datetime(2011, 12, 16), None]])
    collection_extent = pystac.Extent(spatial_extent, temporal_extent)
    
    collection = pystac.Collection(id=collection_id,
                                   title=collection_title,
                                   description=collection_description,
                                   extent=collection_extent)
    
    collection.providers = [
       # pystac.Provider(name='European Space Agency', roles=['producer'], url='https://www.esa.int/'),
       # pystac.Provider(name='European Space Agency', roles=['licensor'], url='https://www.esa.int/'),
        pystac.Provider(name='Satellite Applications Catapult', roles=['processor'], url='https://sa.catapult.org.uk/'),
        pystac.Provider(name='Satellite Applications Catapult', roles=['host'], url='https://sa.catapult.org.uk/')
    ]
    
    obj_paths_list = s3_list_objects_paths(bucket, novasar_dir)
    scene_names = list(np.unique([i.split('/')[2] for i in obj_paths_list if 'uk_novasar_scansar_20m' in i or 'uk_novasar_stripmap_6m' in i]))[1:]
    
    for scene_name in scene_names: #[:n]:
        
        # print(scene_name)
        
        scene_obj_paths = [i for i in obj_paths_list if scene_name in i]
        
        item = novasar_create_item(scene_name, scene_obj_paths)
        
        if len([i for i in scene_obj_paths if '_Gamma0_' in i])==0:
            
            continue
        
        cog_href = create_uri([i for i in scene_obj_paths if '_Gamma0_' in i][0])
        
        if len([i for i in scene_obj_paths if i.endswith('.jpg')])==0: # messy workaround to account for some scenes not having thumbnails
            
            thumb_href = ''
            
        else: 
            
            thumb_href = create_uri([i for i in scene_obj_paths if i.endswith('.jpg')][0])

        asset_hrefs = [cog_href, thumb_href]
        
        add_assets_novasar(item, asset_hrefs) # only works for gamma 0 products (so far)
        
        collection.add_item(item)
    
    collection.update_extent_from_items()
        
    return collection

In [107]:
collection_6m = create_novasar_collection('novasar_uk_test/uk_novasar_stripmap_6m')
collection_20m = create_novasar_collection('novasar_uk_test/uk_novasar_scansar_20m')

In [95]:
#collection_6m.validate()
#collection_20m.validate()

In [96]:
def upload_to_s3(body: bytes, key: str):
    
    s3_client, bucket = s3_create_client(s3_bucket='public-eo-data')

    s3_client.put_object(Bucket='public-eo-data',
                         Key=key,
                         Body=body)

## Overall Catalog

Create catalog for 6m and 20m collections

In [215]:
#export
def novasar_create_catalog(collections):

    catalog_id = 'novasar-overall'
    catalog_title = 'Catapult-hosted Novasar'
    catalog_description = '''### NovaSAR test STACs

    A collection of NovaSAR COGs stored in Catapult object storage
    '''
    # catalog_extensions = [SarExtension.get_schema_uri(), ProjectionExtension.get_schema_uri()]

    catalog = pystac.Catalog(id=catalog_id,
                             title=catalog_title,
                             description=catalog_description)
                             # stac_extensions=catalog_extensions)
    
    catalog.add_children(collections)
    
    # SarExtension.add_to(catalog)

    return catalog

In [208]:
catalog=novasar_create_catalog([collection_6m, collection_20m])

In [209]:
#next(next(catalog.get_children()).get_items()).to_dict()

In [210]:
catalog.to_dict()

{'type': 'Catalog',
 'id': 'novasar-overall',
 'stac_version': '1.0.0',
 'description': '### NovaSAR test STACs\n\n    A collection of NovaSAR COGs stored in Catapult object storage\n    ',
 'links': [{'rel': <RelType.ROOT: 'root'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.CHILD: 'child'>,
   'href': 'http://s3-uk-1.sa-catapult.co.uk/public-eo-data/stac_catalogs/novasar_test/novasar_stripmap_6m/collection.json',
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.CHILD: 'child'>,
   'href': 'http://s3-uk-1.sa-catapult.co.uk/public-eo-data/stac_catalogs/novasar_test/novasar_scansar_20m/collection.json',
   'type': <MediaType.JSON: 'application/json'>}],
 'stac_extensions': [],
 'title': 'Catapult-hosted Novasar'}

In [211]:
catalog.normalize_hrefs(create_uri('stac_catalogs/novasar_test'))

# collection_6m.to_dict()

In [212]:
# os.path.split(catalog.get_self_href())

In [201]:
def save_cat_to_s3(catalog):
    
    upload_to_s3(body=json.dumps(catalog.to_dict(), indent = 2), 
                 key=catalog.get_self_href().split('/', 4)[4])
    
    for collection in catalog.get_children():
    
        upload_to_s3(body=json.dumps(collection.to_dict(), indent = 2), 
                 key=collection.get_self_href().split('/', 4)[4]) 
    
        for item in collection.get_items():

            upload_to_s3(body=json.dumps(item.to_dict(), indent = 2), 
                         key=item.get_self_href().split('/', 4)[4]) 

In [202]:
# save_cat_to_s3(catalog)

In [203]:
next(catalog.get_children()).get_self_href().split('/', 4)[4]

'stac_catalogs/novasar_test/novasar_stripmap_6m/collection.json'

In [214]:
catalog.to_dict()

{'type': 'Catalog',
 'id': 'novasar-overall',
 'stac_version': '1.0.0',
 'description': '### NovaSAR test STACs\n\n    A collection of NovaSAR COGs stored in Catapult object storage\n    ',
 'links': [{'rel': <RelType.ROOT: 'root'>,
   'href': 'http://s3-uk-1.sa-catapult.co.uk/public-eo-data/stac_catalogs/novasar_test/catalog.json',
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.CHILD: 'child'>,
   'href': 'http://s3-uk-1.sa-catapult.co.uk/public-eo-data/stac_catalogs/novasar_test/novasar_stripmap_6m/collection.json',
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.CHILD: 'child'>,
   'href': 'http://s3-uk-1.sa-catapult.co.uk/public-eo-data/stac_catalogs/novasar_test/novasar_scansar_20m/collection.json',
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.SELF: 'self'>,
   'href': 'http://s3-uk-1.sa-catapult.co.uk/public-eo-data/stac_catalogs/novasar_test/catalog.json',
   'type': <MediaType.JSON: 'application/json'>}],
 'stac

In [213]:
catalog.validate_all()

False

## SeDAS 

First need to log in...

In [4]:
from urllib.error import HTTPError

from sedas_pyapi.sedas_api import SeDASAPI

if __name__ == '__main__':

    # creating the SeDASAPI object attempts to log into live. It will throw an exception if it cant. so we feed it real creds (any one using test should have a live set)
    _username = os.getenv('SEDAS_USERNAME')
    __password = os.getenv('SEDAS_PWD')

    # Note the SeDASBulkDownload is very chatty at debug. But if you need to know what is going on enable logging.

    # create the object this will connect to the test.
    sedas = SeDASAPI(_username, __password)

    # set the base url to point at the test instance
    sedas.base_url = "https://geobrowsertest.satapps.org/api/"

    # Now we need to reset a few variables that have the original base url still
    sedas.sensor_url = f"{sedas.base_url}sensors"
    sedas.authentication_url = f"{sedas.base_url}authentication"
    sedas.search_url = f"{sedas.base_url}search" #"https://geobrowsertest.satapps.org/api/ais/search
    

    # Get rid of the token force the log in to happen again.
    sedas._token = None  

    # now we can get the users actual test password
    sedas._username = os.getenv('SEDAS_USERNAME')
    sedas.__password = os.getenv('SEDAS_PWD')
    # and log into test
    sedas.login()


In [86]:
sp_extent = json.dumps(collection_6m.extent.spatial.to_dict())
temp_extent = json.dumps(collection_6m.extent.temporal.to_dict())

NameError: name 'collection_6m' is not defined

In [47]:
left_bounds, bot_bounds, right_bounds, top_bounds = sp_extent.split('[[')[1].split(']]')[0].split(',')

In [48]:
wkt = f"POLYGON(({left_bounds} {bot_bounds}," \
      f"{left_bounds} {top_bounds}," \
      f"{right_bounds} {top_bounds}," \
      f"{right_bounds} {bot_bounds}," \
      f"{left_bounds} {bot_bounds}))"

startDate, endDate = temp_extent.split('[[')[1].split(']]')[0].split(',')

result_sar = sedas.search_sar(wkt, eval(startDate), eval(endDate), satellite_name="NovaSAR-1")

In [5]:
wkt = f"POLYGON((0.263671875 50.28933925329178," \
      f"0.263671875 51.91716758909015," \
      f"2.548828125 51.91716758909015," \
      f"2.548828125 50.28933925329178," \
      f"0.263671875 50.28933925329178))"

result_sar = sedas.search_sar(wkt, '2019-01-01T09:43:45Z', '2021-07-01T09:43:45Z', source_group="NovaSAR")

In [49]:
startDate, endDate = temp_extent.split('[[')[1].split(']]')[0].split(',')

In [50]:
eval(startDate)

'2020-01-29T09:43:45Z'

In [75]:
result_groups = sedas.list_sensor_groups()
# print(json.dumps(result_groups, sort_keys=True, indent=4, separators=(',', ': ')))

In [76]:
# print(json.dumps(result_sar, indent = 4))

In [87]:
wkt = f"POLYGON((0.263671875 50.28933925329178," \
      f"0.263671875 51.91716758909015," \
      f"2.548828125 51.91716758909015," \
      f"2.548828125 50.28933925329178," \
      f"0.263671875 50.28933925329178))"

result = sedas.search(wkt, '2019-01-01T09:43:45Z', '2021-07-01T09:43:45Z')

#### Function to find thumbnails (from SeDAS API) then save ot S3 (NovaSAR)

In [219]:
import re
import requests

def get_thumbs(scene_names):

    for scene_name in scene_names:

        singleProduct = sedas.search_product('_'.join(scene_name.split('_')[0:4] + scene_name.split('_')[5:8])) #searches for scenes

        if singleProduct == []:

            print('no_thumbnail')

        else: 

            print('thumbnail for', scene_name)

            thumb_url = re.findall("https?://[^\s]+", json.dumps(singleProduct, sort_keys=True, indent=4, separators=('"',' ')))[2].replace('""','') #gets thumb url

            tn = requests.get(thumb_url, stream = True).content #saves jpg as bytes

            scene_obj_paths = [i for i in obj_paths_list if scene_name in i]

            upload_to_s3(tn, f'{os.path.dirname(scene_obj_paths[0])}/thumbnail.jpg') #saves to s3

In [221]:
get_thumbs(scene_names[60:])

no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
thumbnail for NovaSAR_01_13113_slc_11_200719_232900_VV_1_ML_TC_TF_cog
no_thumbnail
no_thumbnail
thumbnail for NovaSAR_01_13145_slc_11_200720_233418_HH_1_ML_TC_TF_cog
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
thumbnail for NovaSAR_01_13449_slc_11_200729_225102_HH_ML_TC_TF_cog
thumbnail for NovaSAR_01_13612_slc_11_200803_231937_HH_1_ML_TC_TF_cog
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
thumbnail for NovaSAR_01_14265_slc_11_200822_233314_HH_1_ML_TC_TF_cog
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
thumbnail for NovaSAR_01_14295_slc_11_200823_233848_HH_1_ML_TC_TF_cog
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
no_thumbnail
thumbnail for NovaSAR_01_15228_slc_11_200920_095717_HH_1_ML_TC_TF_cog
no_thumbnail
thumbnail for Nova

In [6]:
os.getenv('SEDAS_USERNAME')

'ofkrauss'

In [70]:
jso = {
  "sourceGroup" : "ALL",
  "polygon" : f"POLYGON((0.263671875 50.28933925329178," \
      f"0.263671875 51.91716758909015," \
      f"2.548828125 51.91716758909015," \
      f"2.548828125 50.28933925329178," \
      f"0.263671875 50.28933925329178))",
  "start" : "2019-04-01T00:00:00Z",
  "stop" : "2021-04-01T00:00:00Z",
  "filters" : {
    "messageType" : "1",
    "mmsi" : "636017265"
  }
}

# request.post( json)

In [80]:
import requests

sedas.search()

TypeError: search() missing 3 required positional arguments: 'wkt', 'start_date', and 'end_date'

In [74]:
type(jso)

dict

In [53]:
from PIL import Image
import numpy

im = Image.open('NovaSAR_01_12547_slc_11_200629_095854_HH_1_Gamma0_Intensity_HH_db.tif')
# im.show()

imarray = numpy.array(im)

In [54]:
imarray

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [48]:
os.listdir()

['.dockerignore',
 '.env.example',
 '.git',
 '.gitignore',
 '.ipynb_checkpoints',
 '00_extensions.ipynb',
 '01A_pleiades.ipynb',
 '01B_pleiades_prep_worker.ipynb',
 '02A_spot.ipynb',
 '03A_novasar.ipynb',
 '04A_cs_sentinel2.ipynb',
 '09_cs_stact_catalog.ipynb',
 '09_sac_stac_catalog.ipynb',
 '10_load_stac.ipynb',
 'docker-compose.yml',
 'Dockerfile',
 'NovaSAR_01_12547_slc_11_200629_095854_HH_1_Gamma0_Intensity_HH_db.tif',
 'NovaSAR_01_13113_slc_11_200719_232900_VV_1_Gamma0_Intensity_VV_db.tif',
 'NovaSAR_01_9629_slc_11_200129_094345_HH_1_Gamma0_Intensity_HH_db.tif',
 'README.md',
 'test.py',
 'utils.ipynb']